
discord remote control?

export TOKENPATH=/Users/briancullinan/.credentials/discord-bot6.txt && export DEFAULT_APPLICATION=1335491680630997034 && npm run import -- "discord remote control"




In [ ]:

const {
  registerCommand, getCommands, deleteCommand, updateCommand
} = importer.import('discord api')
const {interactionsCommands} = importer.import('discord gateway')
const {discordExpress, closeExpress} = importer.import('discord remote proxy server')

const ALL_COMMANDS = [
  'remote',
]

// bot commands using new API, same names as above but lower-case
async function activityCommands(guildId = null) {
  var cmd, cmdDef
  var commandResult = await getCommands(guildId)
  var commands = commandResult.map(command => command.name)

  interactionsCommands['startActivity'] = discordExpress
  interactionsCommands['endActivity'] = closeExpress

  cmdDef = {
    'name': 'remote',
    'description': 'Launch the remote',
    'type': 4,
    'handler': 2,
  }
  if(!commands.includes('remote')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'remote')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }

  var toRemove = commandResult.filter(c => !ALL_COMMANDS.includes(c.name))
  for(var i = 0; i < toRemove.length; i++) {
    await deleteCommand(toRemove[i].id, guildId)
  }

  return await getCommands()
}

module.exports = activityCommands





discord remote proxy server?

https://1335491680630997034.discordsays.com/.proxy/



In [ ]:
const {doClick, getMonitor, getToken} = importer.import('express automation routes')
const {DEFAULT_APPLICATION} = importer.import('discord configuration')

let instances = {}
const express = require('express')
const cors = require('cors')
const app = express()

app.use(cors({
  origin: '*' 
}))
app.use(express.json());
app.post('/click', doClick);
app.get('/monitors/*', getMonitor);
app.post('/api/token', getToken);
app.get('*', serveHomepage);

function serveHomepage(req, res, next) {
  if(req.path.length <= 1) {
    let htmlCode = importer.interpret('remote desktop client').code
    htmlCode = htmlCode.replaceAll('{DEFAULT_APPLICATION}', DEFAULT_APPLICATION)
    return res.send(htmlCode)
  }
  return next()
}

let server

async function discordExpress(activity) {
  if(!express) {
    return false
  }
  
  process.env['NODE_TLS_REJECT_UNAUTHORIZED'] = 0;

  if(activity.participants.length == 0 && server) {
    server.close()
    server = null
  }

  // TODO: check if user was removed
  if(typeof instances[activity['instance_id']] == 'undefined') {
    instances[activity['instance_id']] = []
  }
  instances[activity['instance_id']] = instances[activity['instance_id']].filter(i => activity.participants.includes(i[0]))

  if(activity.length > 1 || server) {
    return
  }

  return (server = await app.listen(3000, () => {
    console.log('Listening on: ', 3000)
  }))
}

function closeExpress(activity) {
  if(activity) {
    if(typeof instances[activity['instance_id']] != 'undefined') {
      instances[activity['instance_id']] = instances[activity['instance_id']].filter(i => activity.participants.includes(i[0]))
    }
    return
  }
  if(server) {
    server.close()
    server = null
  }
}

module.exports = {
  discordExpress,
  closeExpress,
}





express automation routes?


In [ ]:
const {request} = require('gaxios')
const {DEFAULT_API, SECRET, TOKEN, DEFAULT_APPLICATION} = importer.import('discord configuration')
const screenshotMonitor = importer.import('node screenshots')
const mouseMove = importer.import('mouse move')
const qs = require('qs');

async function getToken(req, res, next) {
  //console.log(req.body)
  let result = await request({
    url: DEFAULT_API + 'oauth2/token',
    method: 'POST',
    headers: {
      'Content-Type': 'application/x-www-form-urlencoded',
      //'Authorization': 'Bot ' + TOKEN
    },
    data: qs.stringify({
      client_id: DEFAULT_APPLICATION,
      client_secret: SECRET,
      grant_type: 'authorization_code',
			code: req.body.code,
		})
  })
  //console.log(result)
  res.setHeader('Content-Type', 'application/json')
  return res.send(JSON.stringify({access_token: result.data.access_token}))
}

async function getMonitor(req, res, next) {
  let index = parseInt(req.path.replace(/^\/|\/$/gi, '').split('/')[1])
  if(Number.isNaN(index)) {
    index = 0
  }
  let screenshot = await screenshotMonitor(index)
  if(!screenshot) {
    return next()
  }
  res.setHeader('Content-Type', 'image/png')
  return res.send(screenshot)
}

async function doClick(req, res, next) {
  await mouseMove(req.body.x, req.body.y)
  return res.send('')
}

module.exports = {
  doClick,
  getMonitor,
  getToken,
}



remote desktop client?


In [ ]:
<html>
<head>
  <meta http-equiv="Content-Security-Policy" content="default-src *;
   img-src * 'self' data: https:; script-src 'self' 'unsafe-inline' 'unsafe-eval' *;
   style-src  'self' 'unsafe-inline' *" />
  <style type="text/css">
    html, body {
      padding: 0;
      margin: 0;
      background: rgb(51, 51, 68);
      color: white;
      font-family: Verdana, Geneva, Tahoma, sans-serif;
    }
    
    h1,
    h2,
    h3 {
      color: white;
      padding: 10px;
      margin: 0;
    }
    
    div.container {
      width: auto;
      padding: 20mm 16mm;
      background: #222;
      box-shadow: rgb(0 0 0 / 50%) 2px 3px 10px;
      margin: 20px;
      border-radius: 2px;
      border: 1px solid #666;
      position: relative;
      z-index: 1;
    }
    
    div.livedev {
      position: relative;
      margin: 0 auto;
      max-width: 100%;
      max-height: 460px;
      display:flex;
    }
    
    div.livedev img {
      width: 33.3%;
    }
    
    div.livedev p {
      text-align: center;
      padding-bottom: 60%;
    }
    
    div.livedev iframe {
      height: 100%;
      width: 100%;
      position: absolute;
      top: 0;
      left: 0;
      bottom: 0;
      right: 0;
    }
    
  </style>
</head>
<body>

<div class="container">
  <h2>Live Development</h2>
  <ol class="windows"></ol>
  <script type="application/javascript">
    document.addEventListener('DOMContentLoaded', (evt) => {
      document.querySelector('.livedev').addEventListener('click', (evt) => {
        debugger
        let localX = evt.clientX - evt.currentTarget.offsetLeft
        let localY = evt.clientY - evt.currentTarget.offsetTop
        fetch('https://{DEFAULT_APPLICATION}.discordsays.com/.proxy/click', {
          method: 'POST',
          headers: {
            "Content-Type": "application/json",
          },
          body: JSON.stringify({
            x: evt.layerX / evt.currentTarget.clientWidth,
            y: evt.layerY / evt.currentTarget.clientHeight
          })
        })
      })


      setInterval(() => {
        // TODO: refresh images
        let images = document.querySelectorAll('.livedev > img')
        for(let i = 0; i < images.length; i++) {
          images[i].src += images[i].src.replace(/\?.*/, '') + '?t=' + Date.now()
          images[i].style.backgroundImage = "url('" + encodeURI(images[i].src) + "')"
        }
      }, 5000)
    })

  </script>
  <div class="livedev">
    <img src="https://{DEFAULT_APPLICATION}.discordsays.com/.proxy/monitors/0" />
    <img src="https://{DEFAULT_APPLICATION}.discordsays.com/.proxy/monitors/1" />
    <img src="https://{DEFAULT_APPLICATION}.discordsays.com/.proxy/monitors/2" />
    <!--<p>
      <script>
        document.write(`<iframe width="560" height="315" allow="autoplay" modestbranding="true" src="https://www.youtube.com/embed/live_stream?channel=UCPaZDuwY1sJOb5l-QHm9mDw&origin=${window.location.origin}&controls=0&rel=0&autoplay=1" frameborder="0" allowfullscreen></iframe>`)
      </script>
    </p>-->
  </div>

  <p><a class="nowButton" target="_blank" href="https://www.youtube.com/channel/UCPaZDuwY1sJOb5l-QHm9mDw">
      My YouTube Channel</a></p>
</div>
</body>
</html>

